In [1]:
import pydrake

%matplotlib inline

from IPython.display import display, SVG
import numpy as np
import os
import pydot
import sys

from manipulation.meshcat_cpp_utils import StartMeshcat
from manipulation import running_as_notebook

import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import plotly.express as px

from pydrake.all import ( PiecewisePolynomial, MathematicalProgram, JacobianWrtVariable, eq, Solve)

if running_as_notebook and sys.platform == "linux" and os.getenv("DISPLAY") is None:
    from pyvirtualdisplay import Display
    virtual_display = Display(visible=0, size=(1400, 900))
    virtual_display.start()
    
from project_utils_2 import *




In [2]:
def get_contact_points(plant, plant_context):
    pts = []
    contact_results = plant.get_contact_results_output_port().Eval(plant_context)

    slab_frame = plant.GetBodyByName('slab').body_frame()
    X_WS = slab_frame.CalcPoseInWorld(plant_context)

    for i in range(contact_results.num_point_pair_contacts()):
        contact_info = contact_results.point_pair_contact_info(i)
        body_A = contact_info.bodyA_index()
        body_B = contact_info.bodyB_index()
        point = contact_info.contact_point()
        contact_in_slab_frame = X_WS.inverse() @ point
        pts.append(contact_in_slab_frame)
#         print(contact_in_slab_frame)

    return pts

def run_setup(simulator, end_time, speed = 0.5):
    if running_as_notebook:
        simulator.set_target_realtime_rate(speed)
    simulator.AdvanceTo(end_time if running_as_notebook else 0.1) 

In [3]:
estimated_contact_pts = np.array([
    np.array([ 0.05, -0.05, 0.025]),
    np.array([-0.05, -0.05, 0.025]),
    np.array([ 0.05,  0.05, 0.025]),
    np.array([-0.05,  0.05, 0.025]) 
])

In [4]:
class RobustOptimize:
    def __init__(self, mu1 = 1, mu2 = 1, to_rotate = np.pi, T_opt = 150, finding_friction = False):
        self.gravity = np.array([0, 0, -9.8])
        
        ###########   KUKA
        self.q0_init = [-np.pi/2, -np.pi/2, 0.0, -np.pi/2, 0, 0.0, 0]
        self.q_dot_min = -5*np.ones(7)
        self.q_dot_max = 5*np.ones(7)

        self.q_ddot_min = -5*np.ones(7)
        self.q_ddot_max = 5*np.ones(7)
        self.q_ddot_low = -0*np.ones(7)
        self.q_ddot_upp = 0*np.ones(7)

        self.tau_min = -70.0*np.ones(7)
        self.tau_max = 70.0*np.ones(7)
        self.total_iter = 0

        ##########   OBJECT
        self.m_obj = 0.05
        self.len_obj = 0.1
        self.M_obj = self.m_obj*np.eye(3)
        self.I_obj = self.m_obj*self.len_obj**2/6 * np.eye(3)
        
        ###########  KUKA-OBJECT
        self.mu1 = mu1
        self.mu2 = mu2
        self.mu = 2*self.mu1*self.mu2/(self.mu1+self.mu2)
        self.mu_ = self.mu/np.sqrt(2)
        self.factor = 0.0 # friction safety margin
        self.contact_pts_wrt_slab = [
            np.array([ 0.05, -0.05, 0.025]),
            np.array([-0.05, -0.05, 0.025]),
            np.array([ 0.05,  0.05, 0.025]),
            np.array([-0.05,  0.05, 0.025]) 
        ]
        
        self.box_com_wrt_slab = np.array([0, 0, 0.025+self.len_obj/2])
        
        
        ###########  PLANT
        self.iiwa_diagram, self.iiwa_plant = Iiwa_plant(mu1 = self.mu1)
        self.auto_diff_plant = self.iiwa_plant.ToAutoDiffXd()
        self.auto_diff_context = self.auto_diff_plant.CreateDefaultContext()
        self.iiwa = self.auto_diff_plant.GetModelInstanceByName('iiwa')
        
        
        ########## OPTIMIZATION PARAMETERS
        self.T = T_opt
        self.to_rotate = to_rotate
        self.ss = np.linspace(0, 1, self.T+1)
        self.nq = self.auto_diff_plant.num_positions()
        self.nf = 3 #no of contact forces
        self.delta_s = 1/self.T
        
        ########## OPTIMIZATION FUNCTIONS
        if(not finding_friction):
            self.init_opt_variables()
            self.add_constraints()
            self.set_initial_guess()
        

########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ########### 
        
    def manipulator_equations(self,vars):

        q_k = vars[:7]
        q_dot_k = vars[7:14]
        q_ddot_k = vars[14:14+7]

        f1_k = vars[21:21+3]
        f2_k = vars[24:24+3]
        f3_k = vars[27:27+3]
        f4_k = vars[30:30+3]

        tau_k = vars[33:33+7]

        contact_pt_wrt_slab = np.array([0, 0, -self.len_obj/2])

        self.auto_diff_plant.SetPositions(self.auto_diff_context, self.iiwa, q_k)
        self.auto_diff_plant.SetVelocities(self.auto_diff_context, self.iiwa, q_dot_k)

        M = self.auto_diff_plant.CalcMassMatrixViaInverseDynamics(self.auto_diff_context)
        Cv = self.auto_diff_plant.CalcBiasTerm(self.auto_diff_context)    
        tauG = self.auto_diff_plant.CalcGravityGeneralizedForces(self.auto_diff_context)
        slab_frame = self.auto_diff_plant.GetBodyByName('slab').body_frame()
        ground_frame = self.auto_diff_plant.world_frame()

        # compute Jacobian matrix
        Js = [None]*4
        Js_value = [None]*4
        for i in range(4):
            Js[i] = self.auto_diff_plant.CalcJacobianTranslationalVelocity(
                     self.auto_diff_context,
                     JacobianWrtVariable(0),
                     slab_frame,
                     self.contact_pts_wrt_slab[i],
                     ground_frame,
                     ground_frame)
            
            Js_value[i] = np.array(pydrake.autodiffutils.ExtractValue(self.auto_diff_plant.CalcJacobianTranslationalVelocity(
                     self.auto_diff_context,
                     JacobianWrtVariable(0),
                     slab_frame,
                     self.contact_pts_wrt_slab[i],
                     ground_frame,
                     ground_frame)))
            
        jacob_fric = Js[0].T.dot(f1_k) + Js[1].T.dot(f2_k) + \
                    Js[2].T.dot(f3_k) + Js[3].T.dot(f4_k)
        return (M.dot(q_ddot_k) + Cv - tauG - jacob_fric - tau_k)
    
    
 ########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ###########    
    


    ########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ###########     
    
    def object_equations(self, vars):
        self.total_iter += 1
#         if (self.total_iter%self.T == 0):
#             print("one horizon complete,", self.total_iter/self.T)
#             print("\n")
        q_k = vars[:7]
        q_dot_k = vars[7:14]
        q_ddot_k = vars[14:14+7]
        f1_k = vars[21:21+3]
        f2_k = vars[24:24+3]
        f3_k = vars[27:27+3]
        f4_k = vars[30:30+3]

        self.auto_diff_plant.SetPositions(self.auto_diff_context, self.iiwa, q_k)
        self.auto_diff_plant.SetVelocities(self.auto_diff_context, self.iiwa, q_dot_k)

        slab_frame = self.auto_diff_plant.GetBodyByName('slab').body_frame()
        slab_orientation = slab_frame.CalcPoseInWorld(self.auto_diff_context).rotation()
        ground_frame = self.auto_diff_plant.world_frame()
        
        Js_V_ABp = self.auto_diff_plant.CalcJacobianSpatialVelocity(
                        self.auto_diff_context,
                        JacobianWrtVariable(0),
                        slab_frame,
                        self.box_com_wrt_slab,
                        ground_frame,
                        ground_frame)
        AsBias_ABp = self.auto_diff_plant.CalcBiasSpatialAcceleration(
                        self.auto_diff_context,
                        JacobianWrtVariable(1),
                        slab_frame,
                        self.box_com_wrt_slab,
                        ground_frame,
                        ground_frame)
        
        torques = [None]*4

        pi_slab = self.contact_pts_wrt_slab[0]
        com_slab = self.box_com_wrt_slab
        direction_world = slab_orientation @ (pi_slab - com_slab)
        torques[0] = np.cross(direction_world, f1_k)

        pi_slab = self.contact_pts_wrt_slab[1]
        com_slab = self.box_com_wrt_slab
        direction_world = slab_orientation @ (pi_slab - com_slab)
        torques[1] = np.cross(direction_world, f2_k)

        pi_slab = self.contact_pts_wrt_slab[2]
        com_slab = self.box_com_wrt_slab
        direction_world = slab_orientation @ (pi_slab - com_slab)
        torques[2] = np.cross(direction_world, f3_k)

        pi_slab = self.contact_pts_wrt_slab[3]
        com_slab = self.box_com_wrt_slab
        direction_world = slab_orientation @ (pi_slab - com_slab)
        torques[3] = np.cross(direction_world, f4_k)

        net_torque = torques[0] + torques[1] + torques[2] + torques[3]
        linear_acc_term = self.M_obj.dot(Js_V_ABp.dot(q_ddot_k)[3:] + AsBias_ABp.translational())\
                - (f1_k + f2_k + f3_k + f4_k + self.M_obj.dot(self.gravity))
        ang_acc_term = self.I_obj.dot(Js_V_ABp.dot(q_ddot_k)[:3] + AsBias_ABp.rotational())-net_torque
        return np.concatenate(([linear_acc_term, ang_acc_term]), axis = 0)
    
    
########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ###########     
    
    def get_q(self, s):
        q = self.q0_init.copy()
        q[0] = q[0] + s*self.to_rotate
        return q
        
    
    
    def get_dq_ds(self, q_s):
        dq_ds = np.zeros_like(q_s)
        interval_size = 1/(q_s.shape[0] - 1)
        dq_ds[1:] = (q_s[1:, :] - q_s[:-1, :])/interval_size
        dq_ds[0] = dq_ds[-1]
        return dq_ds
    
    
    def get_d2q_ds2(self, dq_ds):
        l = dq_ds.shape[0]
        nq = 7
        interval_size = 1/(dq_ds.shape[0] - 1)
        d2q_ds2 = np.zeros((l-1, nq))
        d2q_ds2 = (dq_ds[1:, :] - dq_ds[:-1, :])/interval_size
        return d2q_ds2
    
    
########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ########### 


    def init_opt_variables(self):
        self.prog = MathematicalProgram()

        self.q_s = np.array([self.get_q(i) for i in self.ss])
        self.dq_ds = self.get_dq_ds(self.q_s)
        self.d2q_ds2 = self.get_d2q_ds2(self.dq_ds)
        
        self.f1 = self.prog.NewContinuousVariables(rows=self.T, cols=self.nf, name='f1')
        self.f2 = self.prog.NewContinuousVariables(rows=self.T, cols=self.nf, name='f2')
        self.f3 = self.prog.NewContinuousVariables(rows=self.T, cols=self.nf, name='f3')
        self.f4 = self.prog.NewContinuousVariables(rows=self.T, cols=self.nf, name='f4')

        self.S_dot = self.prog.NewContinuousVariables(self.T+1, name = 'S_dot')
        self.S_ddot = self.prog.NewContinuousVariables(self.T, name = 'S_ddot')

        self.q = self.prog.NewContinuousVariables(rows=self.T+1, cols=self.nq, name = 'q')
        self.q_dot = self.prog.NewContinuousVariables(rows=self.T+1, cols=self.nq, name = 'q_dot')
        self.q_ddot= self.prog.NewContinuousVariables(rows=self.T, cols=self.nq, name = 'q_ddot')

        self.tau = self.prog.NewContinuousVariables(rows=self.T, cols=self.nq, name = 'tau')
        self.lin_acc = self.prog.NewContinuousVariables(rows=self.T, cols=3, name = 'lin_acc')
        self.ang_acc = self.prog.NewContinuousVariables(rows=self.T, cols=3, name = 'ang_acc')

        self.total_force = self.prog.NewContinuousVariables(rows=self.T, cols=3, name = 'tot_force')

        
        
########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ########### 

    def cost_func(self, S_dot_var):
            cost = 0
            for i in range(self.T):
                cost = cost + 2*self.delta_s*1/(S_dot_var[i] + S_dot_var[i+1])
            return cost

    def add_constraints(self):
        # S_dot positive constraint   
        for i in range(self.T+1):
            self.prog.AddLinearConstraint(self.S_dot[i] >= 0)

        # Interpolation Constraints
        for i in range(self.T):
            self.prog.AddConstraint(self.S_ddot[i] == (self.S_dot[i+1]**2 - self.S_dot[i]**2)/(2*self.delta_s))

        # q constraint
        for i in range(self.T+1):
            self.prog.AddConstraint(eq(self.q_s[i], self.q[i]))


        # q_dot constraint
        for i in range(self.T+1):
            self.prog.AddConstraint(eq(self.q_dot[i],self.dq_ds[i]*self.S_dot[i]))
            self.prog.AddBoundingBoxConstraint(self.q_dot_min, self.q_dot_max, self.q_dot[i])


        # q_ddot constraint
        for i in range(self.T):
            q_ddot_i = self.d2q_ds2[i]*self.S_dot[i]**2 + self.dq_ds[i]*self.S_ddot[i]
            self.prog.AddConstraint(eq(q_ddot_i, self.q_ddot[i]))
            self.prog.AddBoundingBoxConstraint(self.q_ddot_min - self.q_ddot_low, 
                                          self.q_ddot_max - self.q_ddot_upp, self.q_ddot[i])

        # initial and final constraint
        self.prog.AddConstraint(eq(self.q_dot[0], 0.0))
        self.prog.AddConstraint(eq(self.q_dot[self.T], 0.0))

        # tau constraint
        for i in range(self.T):
            self.prog.AddBoundingBoxConstraint(self.tau_min, self.tau_max, self.tau[i])

        # dynamics constraints
        for t in range(self.T):    
            vars = np.concatenate((self.q[t+1], self.q_dot[t+1], self.q_ddot[t], self.f1[t], 
                                   self.f2[t], self.f3[t], self.f4[t], self.tau[t]))
            self.prog.AddConstraint(self.manipulator_equations, lb=[-1e-6]*7, ub=[1e-6]*7, vars=vars)


        
        lower_limit =  0#1e-2
        
        # friction cone constraints
        for t in range(self.T):
            self.prog.AddConstraint(self.f1[t, 2] >= lower_limit)
            self.prog.AddConstraint(self.f2[t, 2] >= lower_limit)
            self.prog.AddConstraint(self.f3[t, 2] >= lower_limit)
            self.prog.AddConstraint(self.f4[t, 2] >= lower_limit)

            self.prog.AddConstraint(self.f1[t, 0] +self.factor<= self.mu_*self.f1[t, 2])
            self.prog.AddConstraint(self.f2[t, 0] +self.factor<= self.mu_*self.f2[t, 2])
            self.prog.AddConstraint(self.f3[t, 0] +self.factor<= self.mu_*self.f3[t, 2])
            self.prog.AddConstraint(self.f4[t, 0] +self.factor<= self.mu_*self.f4[t, 2])

            self.prog.AddConstraint(self.f1[t, 0] >= self.factor-self.mu_*self.f1[t, 2])
            self.prog.AddConstraint(self.f2[t, 0] >= self.factor-self.mu_*self.f2[t, 2])
            self.prog.AddConstraint(self.f3[t, 0] >= self.factor-self.mu_*self.f3[t, 2])
            self.prog.AddConstraint(self.f4[t, 0] >= self.factor-self.mu_*self.f4[t, 2])


            self.prog.AddConstraint(self.f1[t, 1] +self.factor<= self.mu_*self.f1[t, 2])
            self.prog.AddConstraint(self.f2[t, 1] +self.factor<= self.mu_*self.f2[t, 2])
            self.prog.AddConstraint(self.f3[t, 1] +self.factor<= self.mu_*self.f3[t, 2])
            self.prog.AddConstraint(self.f4[t, 1] +self.factor <= self.mu_*self.f4[t, 2])

            self.prog.AddConstraint(self.f1[t, 1] >= self.factor-self.mu_*self.f1[t, 2])
            self.prog.AddConstraint(self.f2[t, 1] >= self.factor-self.mu_*self.f2[t, 2])
            self.prog.AddConstraint(self.f3[t, 1] >= self.factor-self.mu_*self.f3[t, 2])
            self.prog.AddConstraint(self.f4[t, 1] >= self.factor-self.mu_*self.f4[t, 2])
            
            self.prog.AddConstraint(self.f1[t, 0] == self.f3[t, 0])
            self.prog.AddConstraint(self.f1[t, 1] == self.f3[t, 1])
            self.prog.AddConstraint(self.f2[t, 0] == self.f4[t, 0])
            self.prog.AddConstraint(self.f2[t, 1] == self.f4[t, 1])
            

        # object equations
        for t in range(self.T):
        
            vars = np.concatenate((self.q[t+1], self.q_dot[t+1], self.q_ddot[t], self.f1[t], self.f2[t], 
                                   self.f3[t], self.f4[t]))
            self.prog.AddConstraint(self.object_equations, lb=[-1e-6]*6, ub=[1e-6]*6, vars=vars, description = 'object_constraint'+str(t))


        self.prog.AddCost(self.cost_func, self.S_dot)
        

########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ########### 
       
    def set_initial_guess(self):
        lst1 = [0]*self.nq
        lst2 = [0]*self.nq
        dq_ds_min = np.abs(np.amin(self.dq_ds, axis = 0))
        dq_ds_max = np.abs(np.amax(self.dq_ds, axis = 0))
        dq_ds_min_lb = [0]*self.nq
        dq_ds_max_lb = [0]*self.nq
        for i in range(self.nq):
            dq_ds_min_lb[i] = max(dq_ds_min[i], 1e-5)
            dq_ds_max_lb[i] = max(dq_ds_max[i], 1e-5)

            lst1[i] = np.abs(self.q_dot_max[i]/dq_ds_max_lb[i])
            lst2[i] = np.abs(self.q_dot_min[i]/dq_ds_min_lb[i])

        s_dot_0 = min(min(lst1), min(lst2))


        self.initial_guess = np.zeros(self.prog.num_vars())
        self.prog.SetDecisionVariableValueInVector(self.S_dot, [s_dot_0]*(self.T+1), self.initial_guess)
        self.prog.SetDecisionVariableValueInVector(self.S_ddot, [0]*self.T, self.initial_guess)

        self.prog.SetDecisionVariableValueInVector(self.q, self.q_s, self.initial_guess)
        self.prog.SetDecisionVariableValueInVector(self.q_dot, self.dq_ds*s_dot_0, self.initial_guess)

        q_ddot_guess = self.d2q_ds2*s_dot_0**2
        self.prog.SetDecisionVariableValueInVector(self.q_ddot,  q_ddot_guess, self.initial_guess)

        self.prog.SetDecisionVariableValueInVector(self.f1, 0.01*np.ones((self.T, self.nf)), self.initial_guess)
        self.prog.SetDecisionVariableValueInVector(self.f2, 0.01*np.ones((self.T, self.nf)), self.initial_guess)
        self.prog.SetDecisionVariableValueInVector(self.f3, 0.01*np.ones((self.T, self.nf)), self.initial_guess)
        self.prog.SetDecisionVariableValueInVector(self.f4, 0.01*np.ones((self.T, self.nf)), self.initial_guess)
        
########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ########### 
        
        
    def solve_optimization(self):
        self.result = Solve(self.prog, initial_guess = self.initial_guess)
        # solver = SnoptSolver()
        # result = solver.Solve(prog, initial_guess)

        if self.result.is_success():
            assert True
#             print("Total Time: ",self.result.get_optimal_cost())
        else:
            print("Optimization Failed !!!")
            
        return self.result

            
            
            
 ########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ###########            

########### ########### ########### ########### ########### ########### ########### ########### 
########### ########### ########### ########### ########### ########### ########### ########### 
    
    def get_trajectories(self, s_dot_new = None, num_steps = 200):
        T_new = num_steps
        s_new = np.linspace(0, 1, T_new+1)
        S_dot_val = self.result.GetSolution(self.S_dot)
        S_ddot_val = self.result.GetSolution(self.S_ddot)
        
        if (s_dot_new is None):
            S_dot_new = np.interp(s_new, self.ss, S_dot_val)
        else:
            S_dot_new = np.interp(s_new, self.ss, s_dot_new)
        S_ddot_new = np.interp(np.linspace(0, 1, T_new), self.ss[:-1], S_ddot_val)
        
        t_val = np.zeros(T_new+1)

        for k in range(1, T_new+1):
            t_val[k] = t_val[k-1] + (2*1/T_new)/(S_dot_new[k] + S_dot_new[k-1])

        qs = np.array([self.get_q(i) for i in s_new])
        
        #velocity
        qs_dot = np.zeros_like(qs)
        dq_ds = self.get_dq_ds(qs)
        for i in range(T_new+1):
            qs_dot[i] = dq_ds[i]*S_dot_new[i]
            
        #acceleration
        qs_ddot = np.zeros((T_new,7))
        d2q_ds2 = self.get_d2q_ds2(dq_ds)
        for i in range(T_new):
            qs_ddot[i] = d2q_ds2[i]*(S_dot_new[i])**2 + dq_ds[i]*S_ddot_new[i] 
        
        
        qs_new= np.insert(qs, 0, qs[0,:], axis = 0)
        qdot_new= np.insert(qs_dot, 0, qs_dot[0,:], axis = 0)
        #acceleration
        qddot_new= np.insert(qs_ddot, 0, np.zeros(7), axis = 0)
#         qddot_new= np.insert(qddot_new, -1, qddot_new[-1,:], axis = 0)
    
        
        ts = np.zeros(T_new+2)
        ts[1:] = 1.0 + np.array(t_val)
        
        #acceleration
        ts_ddot = np.zeros(T_new+1)
        ts_ddot[1:] = 1.0 + np.array(t_val[:-1])

        q_traj = PiecewisePolynomial.FirstOrderHold(ts, qs_new.T)   
        q_dot_traj = PiecewisePolynomial.FirstOrderHold(ts, qdot_new.T)
        q_ddot_traj = PiecewisePolynomial.ZeroOrderHold(ts_ddot, qddot_new.T)
        
        return q_traj, q_dot_traj, q_ddot_traj, ts
    
    def plotting(self):
        q_traj, q_dot_traj, _, _ = self.get_trajectories()
        data = dataframe(q_traj, q_traj.get_segment_times(), ['q1','q2', 'q3', 'q4', 'q5', 'q6', 'q7'])
        alt.vconcat(alt.Chart(data).mark_line().encode(x='t', y='q1').properties(height=80),)
        
        data = dataframe(q_dot_traj, q_dot_traj.get_segment_times(), ['q1_dot','q2_dot', 'q3_dot', 'q4_dot', 'q5_dot', 'q6_dot', 'q7_dot'])
        alt.vconcat(alt.Chart(data).mark_line().encode(x='t', y='q1_dot').properties(height=80),)
        
    
    def debug_friction(self):
        lin_acc_val = self.result.GetSolution(self.lin_acc)

        q_ddot_val = self.result.GetSolution(self.q_ddot)
        q_dot_val = self.result.GetSolution(self.q_dot)
        total_force_val = self.result.GetSolution(self.total_force)
        q_val = self.result.GetSolution(self.q)
        q_val[:,0]+=np.pi


        theta_cap = 0.3995*q_ddot_val[:,0]*0.05
        r_cap = -0.3995*q_dot_val[1:,0]**2*0.05
        f_x = r_cap*np.cos(q_val[1:,0])-theta_cap*np.sin(q_val[1:,0])
        f_y = r_cap*np.sin(q_val[1:,0])+theta_cap*np.cos(q_val[1:,0])


        print(f_x - total_force_val[:,0], f_y - total_force_val[:,1])
        
        
    def debug_friction_norm(self, mu = None):
        if isinstance(mu, type(None)):
            mu = self.mu
        
        
        total_force_val = self.result.GetSolution(self.total_force)
        f1 = self.result.GetSolution(self.f1)
        f2 = self.result.GetSolution(self.f2)
        f3 = self.result.GetSolution(self.f3)
        f4 = self.result.GetSolution(self.f4)
            
            
        error_f1 = (mu*f1[:,2])**2 - (f1[:,0]**2 + f1[:,1]**2) + 1e-6
        error_f2 = (mu*f2[:,2])**2 - (f2[:,0]**2 + f2[:,1]**2) + 1e-6
        error_f3 = (mu*f3[:,2])**2 - (f3[:,0]**2 + f3[:,1]**2) + 1e-6
        error_f4 = (mu*f4[:,2])**2 - (f4[:,0]**2 + f4[:,1]**2) + 1e-6
    
        
        a1 = error_f1 < (2*self.factor**2+2*self.factor*f1[:,0]+2*f1[:,1]*self.factor)
        a2 = error_f2 < (2*self.factor**2+2*self.factor*f2[:,0]+2*f2[:,1]*self.factor)
        a3 = error_f3 < (2*self.factor**2+2*self.factor*f3[:,0]+2*f3[:,1]*self.factor)
        a4 = error_f4 < (2*self.factor**2+2*self.factor*f4[:,0]+2*f4[:,1]*self.factor)
                
        if(sum(a1)+sum(a2)+sum(a3)+sum(a4))>0:
            return False
        else:
            return True
      
        
    def convert_from_expr(self, x):
        x_eval = np.zeros_like(x)
        rows = len(x)
        cols = len(x[0])
        for i in range(rows):
            for j in range(cols):
                x_eval[i,j] = x[i,j].Evaluate()
                
        return x_eval
    
    
    def find_fric_from_traj(self, q, q_dot, q_ddot, true_mu):
        
        time_steps = len(q_ddot)
        total_fric = np.zeros((len(q_ddot),3))
        
        for k in range(time_steps):
            q_k = q[k+1]
            q_dot_k = q_dot[k+1]
            q_ddot_k = q_ddot[k]
            
            self.auto_diff_plant.SetPositions(self.auto_diff_context, self.iiwa, q_k)
            self.auto_diff_plant.SetVelocities(self.auto_diff_context, self.iiwa, q_dot_k)

            slab_frame = self.auto_diff_plant.GetBodyByName('slab').body_frame()
            slab_orientation = slab_frame.CalcPoseInWorld(self.auto_diff_context).rotation()
            ground_frame = self.auto_diff_plant.world_frame()

            Js_V_ABp = self.auto_diff_plant.CalcJacobianSpatialVelocity(
                            self.auto_diff_context,
                            JacobianWrtVariable(0),
                            slab_frame,
                            self.box_com_wrt_slab,
                            ground_frame,
                            ground_frame)
            AsBias_ABp = self.auto_diff_plant.CalcBiasSpatialAcceleration(
                            self.auto_diff_context,
                            JacobianWrtVariable(1),
                            slab_frame,
                            self.box_com_wrt_slab,
                            ground_frame,
                            ground_frame)

            lin_acc = Js_V_ABp.dot(q_ddot_k)[3:] + AsBias_ABp.translational()
            fric = self.M_obj.dot(lin_acc) - self.M_obj.dot(self.gravity)
            fric = pydrake.autodiffutils.ExtractValue(fric)
            total_fric[k,:] = fric.T[0]
            
        a = (total_fric[:,2]*true_mu)**2 - (total_fric[:,0]**2+total_fric[:,1]**2)
        b = a<0
        if(sum(b)>0):
            return False
        else:
            return True
        
    def simulate_trajectory(self, meshcat, q_traj, q_dot_traj, mu1, mu2):
        
        diagram, plant, scene_graph, plant_context = MyManipulationStation(mu1 = mu1, mu2 = mu2)
        simulator, plant_context, q_traj, q_dot_traj_ = ExtendedStation(diagram, plant, meshcat, q_traj, q_dot_traj)   

        run_setup(simulator, q_traj.end_time(), speed = 0.25)
        
        pts = get_contact_points(plant, plant_context)
        if len(pts) < 4:
            return False
        difference = np.abs(pts - estimated_contact_pts)
        if np.any(difference[:,:2] < 0.01):
            return True
        else:
            return False


# Iterative Learning Uncertainties

### Bisection Search on Friction Coefficient


In [5]:
true_mu = 0.1
epsilon = 0.05
T_opt = 10
def bisection_search(mu_init = 1):
    mu_low = 0.05
    mu_up = mu_init
    thresh = 0.01
    
    while ((mu_up-mu_low)>thresh):
        mu_avg = (mu_up+mu_low)/2
        optimizer = RobustOptimize(mu1 = mu_avg, mu2 = mu_avg, to_rotate = np.pi, T_opt = T_opt)
        res = optimizer.solve_optimization() #get trajectory

        q_traj , q_dot_traj, _, _ = optimizer.get_trajectories(num_steps = 100)
        success = optimizer.simulate_trajectory(meshcat, q_traj, q_dot_traj, mu1 = true_mu, mu2 = true_mu)
        print(f'mu_low: {np.round(mu_low, 3)}, mu_up: {np.round(mu_up, 3)}, mu_avg: {np.round(mu_avg, 3)}, Time: {np.round(res.get_optimal_cost(), 3)}, success: {success}')
        
#         success = optimizer.debug_friction_norm(true_mu) #Execute trajectory
        
        if success:
            original_time = res.get_optimal_cost()
            original_S_dot = res.GetSolution(optimizer.S_dot)
            original_S_ddot = res.GetSolution(optimizer.S_ddot)
              
            original_q = res.GetSolution(optimizer.q) 
            if not isinstance(original_q[0,0],(np.floating, float)):
                original_q = optimizer.convert_from_expr(original_q)
            
            original_dq_ds = res.GetSolution(optimizer.dq_ds)
            if not isinstance(original_dq_ds[0,0],(np.floating, float)):
                original_dq_ds = optimizer.convert_from_expr(original_dq_ds)
            
            original_d2q_ds2 = res.GetSolution(optimizer.d2q_ds2)
            if not isinstance(original_d2q_ds2[0,0],(np.floating, float)):
                original_d2q_ds2 = optimizer.convert_from_expr(original_d2q_ds2)
            
            new_time = original_time/(1+epsilon) #speed up trajectory
            new_S_dot = original_S_dot*(1+epsilon)
            
            new_q_dot = np.array(original_dq_ds)*np.repeat(new_S_dot.reshape((len(new_S_dot),1)), 7, axis = 1)
            if(np.any(new_q_dot<optimizer.q_dot_min) or np.any(new_q_dot>optimizer.q_dot_max)): 
                return mu_avg, False #this means true mu >= mu_avg
            
            new_S_ddot = (new_S_dot[1:]**2 - new_S_dot[:-1]**2)/(2*optimizer.delta_s)
            term1 = original_d2q_ds2*np.repeat(new_S_dot.reshape((len(new_S_dot),1))[:-1,:], 7, axis = 1)
            term2 = original_dq_ds[:-1,:]*np.repeat(new_S_ddot.reshape((len(new_S_ddot),1)), 7, axis = 1)
            new_q_ddot = term1 + term2
            if(np.any(new_q_ddot<optimizer.q_ddot_min) or np.any(new_q_ddot>optimizer.q_ddot_max)):
                return mu_avg, False #this means true mu >= mu_avg
        
            q_traj_new , q_dot_traj_new, _, _ = optimizer.get_trajectories(s_dot_new = new_S_dot, num_steps = 100)            
            success_dash = optimizer.simulate_trajectory(meshcat, q_traj, q_dot_traj, mu1 = true_mu, mu2 = true_mu)
            print(f'    Attempted speedup, success: {success_dash}')

            if(success_dash != success):
                print(f'Returning {mu_low}')
                return mu_low, True
            else:
                mu_low = mu_avg
        
        else:
            mu_up = mu_avg
    
    print(f'Returning {mu_low}')
    return mu_low, True

In [6]:
meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7000


In [7]:
mu, converge = bisection_search()

mu_low: 0.05, mu_up: 1, mu_avg: 0.525, Time: 1.656, success: False
mu_low: 0.05, mu_up: 0.525, mu_avg: 0.288, Time: 1.923, success: False
mu_low: 0.05, mu_up: 0.288, mu_avg: 0.169, Time: 2.448, success: False
mu_low: 0.05, mu_up: 0.169, mu_avg: 0.109, Time: 3.006, success: False
mu_low: 0.05, mu_up: 0.109, mu_avg: 0.08, Time: 3.509, success: True
    Attempted speedup, success: True
mu_low: 0.08, mu_up: 0.109, mu_avg: 0.095, Time: 3.227, success: True
    Attempted speedup, success: True
mu_low: 0.095, mu_up: 0.109, mu_avg: 0.102, Time: 3.11, success: False
Returning 0.09453125
